In [1]:
import torch

from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)
pd.options.display.max_rows = 7


In [2]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])

In [3]:
ratings.shape

(100000, 4)

In [4]:
n_users = len(ratings["user"].unique())
n_items = len(ratings["movie"].unique())

In [5]:
dls = CollabDataLoaders.from_df(ratings, item_name='movie', bs=64)

In [6]:
dls.show_batch()

,user,movie,rating
0,94,451,4
1,897,77,4
2,655,66,2
3,749,233,5
4,393,95,4
5,791,748,3
6,548,117,4
7,919,19,4
8,537,718,4
9,269,55,4


In [7]:
EMB_SIZE = get_emb_sz(dls)
user_embeddings = torch.randn(EMB_SIZE[0])
item_embeddings = torch.randn(EMB_SIZE[1])

In [9]:
# always retrain our model for the first 10 predictions or so

In [28]:
class DotProduct(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5)):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        items = self.item_factors(x[:,1])
        return sigmoid_range((users * items).sum(dim=1), *self.y_range)

In [26]:
embs = get_emb_sz(dls)
model = DotProduct(*embs)

In [29]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time


NameError: name 'movies' is not defined

In [11]:
embs = get_emb_sz(dls)
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [12]:
model = CollabNN(*embs)

In [13]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.937064,0.954076,00:09
1,0.880837,0.906038,00:10
2,0.855758,0.886094,00:10
3,0.790897,0.870600,00:13
4,0.788192,0.874529,00:15


In [22]:
learn.model

CollabNN(
  (user_factors): Embedding(944, 74)
  (item_factors): Embedding(1640, 101)
  (layers): Sequential(
    (0): Linear(in_features=175, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [20]:
learn.predict()

TypeError: list indices must be integers or slices, not list

In [14]:
help(learn)

Help on Learner in module fastai.learner object:

class Learner(fastcore.basics.GetAttr)
 |  Learner(dls: 'DataLoaders', model: 'callable', loss_func: 'callable | None' = None, opt_func: 'Optimizer | OptimWrapper' = <function Adam at 0x7f4a0d91fa30>, lr: 'float | slice' = 0.001, splitter: 'callable' = <function trainable_params at 0x7f4a0fb66d40>, cbs: 'Callback | MutableSequence | None' = None, metrics: 'callable | MutableSequence | None' = None, path: 'str | Path | None' = None, model_dir: 'str | Path' = 'models', wd: 'float | int | None' = None, wd_bn_bias: 'bool' = False, train_bn: 'bool' = True, moms: 'tuple' = (0.95, 0.85, 0.95), default_cbs: 'bool' = True)
 |  
 |  Group together a `model`, some `dls` and a `loss_func` to handle training
 |  
 |  Method resolution order:
 |      Learner
 |      fastcore.basics.GetAttr
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, event_name)
 |      Call `event_name` for all `Callback`s in `self.cbs`
 |  
 |  __e